In [46]:
# QUISPE SOLIZ MATIAS FERNANDO ----- INGENIERIA DE SISTEMAS ----- EXAMEN FINAL SIS420

# Pregunta 2: Implementar una red neuronal, que permita realizar una regresión o clasificación de un problema que no podría ser resuelto por un simple perceptrón.

# ---- Problema: Dataset multiclase , para clasificar el puntaje de crédito de los clientes de una compañia financiera ----

# Cálculo científico y vectorial para python
import numpy as np
# Para manejo y analisis de estructuras de datos (En este caso para leer nuestro dataset de una manera mas comoda)
import pandas as pd


# Leemos nuestro dataset credit_score.csv (Dataset multiclase para predecir el puntaje o calificación de crédito
# de un cliente en una compañia financiera, podria ser un banco por ejemplo)
data = pd.read_csv('credit_score.csv', sep=',')
# Imprimimos para observar los datos que contiene nuestro dataset
print(data.head())


   Unnamed: 0  Month        Age  Occupation  Annual_Income  \
0           1      2  23.000000          13       19114.12   
1           2      6  34.429817          13       19114.12   
2           3      0  23.000000          13       19114.12   
3           4      7  23.000000          13       19114.12   
4           5      5  23.000000          13       19114.12   

   Monthly_Inhand_Salary  Num_Bank_Accounts  Num_Credit_Card  Interest_Rate  \
0            4194.170850                3.0              4.0            3.0   
1            4194.170850                3.0              4.0            3.0   
2            4194.170850                3.0              4.0            3.0   
3            1824.843333                3.0              4.0            3.0   
4            4194.170850                3.0              4.0            3.0   

   Num_of_Loan  ...  Credit_Mix  Outstanding_Debt  Credit_Utilization_Ratio  \
0          4.0  ...           1            809.98                 31.9449

In [47]:
# Como podemos observar el dataset cuenta con una columna sin nombre, lo que nos dara problemas a la hora de trabajar con el dataset
# por lo que optamos por quitar dicha columna, ya que tampoco es de relevancia en dicho dataset porque solo es un ID
# ---Eliminamos la columna---
data = data.drop('Unnamed: 0', axis=1)
print(data.head())



   Month        Age  Occupation  Annual_Income  Monthly_Inhand_Salary  \
0      2  23.000000          13       19114.12            4194.170850   
1      6  34.429817          13       19114.12            4194.170850   
2      0  23.000000          13       19114.12            4194.170850   
3      7  23.000000          13       19114.12            1824.843333   
4      5  23.000000          13       19114.12            4194.170850   

   Num_Bank_Accounts  Num_Credit_Card  Interest_Rate  Num_of_Loan  \
0                3.0              4.0            3.0          4.0   
1                3.0              4.0            3.0          4.0   
2                3.0              4.0            3.0          4.0   
3                3.0              4.0            3.0          4.0   
4                3.0              4.0            3.0          4.0   

   Type_of_Loan  ...  Credit_Mix  Outstanding_Debt  Credit_Utilization_Ratio  \
0           128  ...           1            809.98                

In [48]:
# Ahora podemos observar que el dataset esta listo, para poder asignar las características a X y los valores a predecir a Y
# Convertimos el DataFrame de Pandas en un arreglo de tipo NumPy para poder manejar comodamente
data = data.to_numpy()
# Asignamos los valores correspondientes a las variables X y Y, usando 70000 datos de nuestro dataset, tomando desde el inicio hasta 70000
X, Y = (data[:70000, :23], data[:70000, 23])
print(X)
print(Y)


# Normalizamos los datos haciendo uso de la media y la desviación estándar
X_mean, X_std = X.mean(axis=0), X.std(axis=0)
X_norm = (X - X_mean) / X_std
# Verificamos que tengan las mismas dimensiones
X.shape, Y.shape

[[  2.          23.          13.         ... 118.28022162   3.
  284.6291625 ]
 [  6.          34.42981709  13.         ...  81.69952126   4.
  331.20986285]
 [  0.          23.          13.         ... 199.45807439   5.
  223.45130973]
 ...
 [  5.          49.           4.         ... 637.41299841   1.
  285.71779825]
 [  4.          49.           4.         ...  30.94203438   5.
  304.82771624]
 [  1.          50.           4.         ...  36.97395846   5.
  298.79579216]]
[2. 2. 2. ... 2. 1. 1.]


((70000, 23), (70000,))

In [49]:
# Como ya tenemos listo nuestro dataset para ser utilizado procedemos con el siguiente paso

# Haremos uso de una red neuronal para nuestro modelo

# Primero definimos nuestra clase de Perceptrón Multicapa o MLP por sus siglas en ingles Multi Layer Perceptron
# que nos servirá para resolver nuestro problema de clasificación multiclase de puntaje o calificación de credito de clientes de una compañia financiera

# La clase estará formada por una lista de capas (layers)
# También con la seccion de "for layer y self.layers:" nos encargaremos de la salida del modelo aplicando cada capa de manera secuencial recorriendo todas las capas
class MLP: # Perceptrón Multicapa
    def __init__(self, layers):
        # el Perceptrón Multicapa es una lista de capas
        self.layers = layers

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [50]:
# Con esta clase definimos las diferentes capas que usaremos para nuestro modelo
# Para empezar tendremos una clase base donde estaran los parametro entrenables de la capa en cuestion, asi como tambien sus gradientes

class Layer():
    def __init__(self):
        # Lista de parametros con todos los parametros de la capa
        self.params = []
        # Lista de gradientes que tendremos que aplicar a todos los parametros de la capa
        self.grads = []

    # por defecto, devolver los inputs pero cada capa hará algo diferente aquí
    def __call__(self, x):
        return x

    # Recibe gradientes de las capas de arriba
    # Con estos calcula sus propios gradientes y pasara los gradientes obtenidos a la capa de abajo
    def backward(self, grad):
        return grad

    # Actualizara parametros si los hay, segun lo que diga el optimizador
    def update(self, params):
        return

In [51]:
# Capa con parametros - Capa lineal
# Esta clase representa la capa lineal (Perceptrón)
# Lo usaremos para crear la secuencia de perceptrones (Perceptrón Multicapa)
class Linear(Layer):
    def __init__(self, d_in, d_out):
        # Pesos de la capa (matriz que tendra dimensiones de entrada (d_in) y dimensiones de salida (d_out))
        # Multiplicaremos estos pesos por los inputs
        self.w = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(d_in+d_out)),
                                  size=(d_in, d_out))
        # Parámetro ballas
        self.b = np.zeros(d_out)

    def __call__(self, x):
        self.x = x
        self.params = [self.w, self.b]
        # Salida del preceptrón
        # Producto de la matriz de inputs multiplicado por los pesos, mas el parametro ballas
        return np.dot(x, self.w) + self.b

    # Calculamos las derivadas de la funcion de costo con respecto a los parametros que vamos cambiando
    def backward(self, grad_output):
        # Gradientes para la capa siguiente (BACKPROP) obtenido del producto por los pesos por el gradiente que nos viene de la capa siguiente
        grad = np.dot(grad_output, self.w.T)
        # Gradientes de los pesos obtenido por los inputs por el gradiente de la capa siguiente
        self.grad_w = np.dot(self.x.T, grad_output)
        # Gradientes para actualizar pesos (Gradiente de ballas)
        self.grad_b = grad_output.mean(axis=0)*self.x.shape[0]
        self.grads = [self.grad_w, self.grad_b]
        return grad

    def update(self, params):
        self.w, self.b = params

In [52]:
# Esta capa va a ser una funcion de activacion

class ReLU(Layer):
    def __call__(self, x):
        self.x = x
        # Devuelve el valor maximo entre los inputs que recibe y 0
        return np.maximum(0, x)
    # Calculamos su derivada
    def backward(self, grad_output):
        # Derivada de la ReLU
        grad = self.x > 0
        # Usamos el gradiente de la capa siguiente y lo multiplicamos por la derivada de la ReLU
        return grad_output*grad

# Devuelve valores entre 0 y 1
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

# Los valores mas grandes los hace mas grandes en comparacion a los mas pequeños que los hace mas pequeños con relación al más grande
def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

class Sigmoid(Layer):
    def __call__(self, x):
        self.x = x
        return sigmoid(x)

    def backward(self, grad_output):
        grad = sigmoid(self.x)*(1 - sigmoid(self.x))
        return grad_output*grad

In [53]:
# El optimizador que utilizaremos para nuestro modelo es el Stochastic Gradient Descent
# Este optimizador calcula la derivada para cada elemento del dataset de manera independiente

class SGD():
    # En el inicializador le pasamos la red (net) y el learning rate (lr)
    def __init__(self, net, lr):
        self.net = net
        self.lr = lr

    def update(self):
        # Iteramos por todas las capas de nuestra red neuronal
        for layer in self.net.layers:
            # En cada capa llamaremos a su propia funcion update
            layer.update([
                params - self.lr*grads
                for params, grads in zip(layer.params, layer.grads)
            ])

In [54]:
# En esta clase definiremos nuestras funciones de perdida
# Para nuestro caso como es un problema de clasificación multiclase
# Haremos uso de de la funcion de perdida CrossEntropy


# Clase base de funcion de perdida: Loss
class Loss():
    def __init__(self, net):
        # Obtiene la red neuronal
        self.net = net
    # Desencadena todo el calculo de las derivadas
    def backward(self):
        # derivada de la loss function con respecto a la salida del Perceptron Multicapa
        grad = self.grad_loss()
        # BACKPROPAGATION
        # Iteramos desde la ultima capa hasta llegar a la primera capa, hasta llegar a las entradas
        # Propagando los gradientes desde la ultima capa hasta la primera
        for layer in reversed(self.net.layers):
            grad = layer.backward(grad)

class CrossEntropy(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target
        logits = output[np.arange(len(output)), target.astype(int)]
        loss = - logits + np.log(np.sum(np.exp(output), axis=-1))
        loss = loss.mean()
        # Calculamos y devolvemos una loss (perdida)
        return loss

    # Cuanto necesitemos la derivada calcularemos gradiente de la funcion de perdida
    def grad_loss(self):
        answers = np.zeros_like(self.output)
        answers[np.arange(len(self.output)), self.target.astype(int)] = 1
        return (- answers + softmax(self.output)) / self.output.shape[0]

In [55]:
# En esta parte especificamos los valores para que nuestro modelo entre de la manera adecuada
# Podemos ajustar los valores de acuerdo a nuestro problema especifico, (clasificación multiclase de puntaje o calificación de credito de clientes de una compañia financiera)

# Por ejemplo con D_in = X_norm.shape[1] estoy definiendo las dimensiones de entrada del modelo, en este caso son 23
# Hacemos esto de manera dinamica, ya que obtiene el numero de columnas que tiene X_norm
D_in = X_norm.shape[1]
# Aqui especificamos la cantidad de neuronas que tendrán nuestras capas ocultas, podemos ir probando con diferentes valores
H = 200
# Aqi especificamos nuestra salida como 3 debido a que nuestro problema de clasificacion multiclase, contiene 3 clases diferentes
D_out = 3

# Aqui definimos nuestro Perceptrón Multicapa con las capas que querramos utilizar
mlp = MLP([
    # Capa oculta lineal
    Linear(D_in, H),
    # Funcion de activacion ReLU
    ReLU(),
    # Capa oculta lineal
    Linear(H, H),
    # Funcion de activacion ReLU
    ReLU(),
    # Capa oculta lineal
    Linear(H, H),
    # Funcion de activacion ReLU
    ReLU(),
    # Capa oculta lineal
    Linear(H, H),
    # Funcion de activacion ReLU
    ReLU(),
    # Capa de salida
    Linear(H, D_out)
])

# Aqui definimos el optimizador, recordamos que estamos usando Stochastic Gradient Descent para nuestra optimizacion
# le pasamos nuestra red neuronal definida como mlp (Perceptron Multicapa) y tambien la tasa de aprendizaje lr con un valor de 0.03 (learning rate)
optimizer = SGD(mlp, lr=0.03)
# Declaramos la funcion de perdida como CrossEntropy debido a que estamos frente a un problema de clasificación multiclase
# de puntaje o calificación de credito de clientes de una compañia financiera, para ello le pasamos nuestra red neuronal
loss = CrossEntropy(mlp)

# Los epochs son el numero de interaciones que se llevaran a cabo
# Podemos ir ajustando estos valores de acuerdo a nuestro problema
epochs = 300
# Especificamos antes el bath_size
batch_size = 20

# Aqui se especifica la cantidad del grupo de datos que se tomaran en cuenta
batches = len(X) // batch_size
# El log_each sirve para mostrarnos los avances de nuestro modelo cada tanto (valor de log_each) dentro de los epochs (iteraciones)
log_each = 50
l = []
for e in range(1,epochs+1):
    _l = []
    for b in range(batches):
        x = X_norm[b*batch_size:(b+1)*batch_size]
        y = Y[b*batch_size:(b+1)*batch_size]
        y_pred = mlp(x)
        _l.append(loss(y_pred, y))
        loss.backward()
        optimizer.update()
    l.append(np.mean(_l))
    # Con esto podemos ver como es que va avanzando nuestro modelo mientras entrena
    if not e % log_each:
        print(f'Epoch {e}/{epochs}, Loss: {np.mean(l):.4f}')

# Adjunto datos de prueba y resultados que obtuve para poder llegar a una perdida mas aceptable y un porcentaje de relacion obtenido en la siguiente celda despues de entrenar el modelo
# Se probo cambiando varios valores uno a la vez para poder llegar al mejor conjunto de valores para dicho modelo de redes neuronales
# Se logro deducir por ejemplo que:
# Una tasa de aprendizaje muy baja no es conveniente, y tampoco es conveniente una mas alta
# El valor mas ideal para este modelo es de lr=0.03
# Se logro deducir tambien que mientrar mas neuronas hay en cada capa oculta mejor entrena el modelo
# lo mismo con los epochs(epochs) mientras mas iteraciones se obtiene un mejor resultado pero puede llegar a tardar mucho cuando se exagera
# el batch_size tambien llego a influenciar, ya que afecta un poco a la funcion de perdida y el procentaje de relacion, se llego a un valor ideal de batch_size=20

# Tambien influyo bastante el hecho de agregar mas capas a nuestro modelo de Perceptron multicapa, gracias a ello se logro un mejor resultado.

# Con 300 neuronas, 500 epochs y 50 de batch_size logre una perdida de 13% y un porcentaje de relacion de 100%
# Con 100 neuronas, 100 epochs y 10 de batch_size logre una perdida de 56,07% y un porcentaje de relacion de 84.88%
# Con 100 neuronas, 100 epochs y 10 de batch_size (lr=0.01) logre una perdida de 58,05% y un porcentaje de relacion de 86.56%
# Con 100 neuronas, 100 epochs y 10 de batch_size (lr=0.03) logre una perdida de 55,91% y un porcentaje de relacion de 87.14%
# Con 150 neuronas, 100 epochs y 10 de batch_size (lr=0.02) logre una perdida de 47,24% y un porcentaje de relacion de 93.46%
# Con 150 neuronas, 100 epochs y 10 de batch_size (lr=0.03) logre una perdida de 47,21% y un porcentaje de relacion de 92.56%
# Con 150 neuronas, 100 epochs y 10 de batch_size (lr=0.05) logre una perdida de 51,42% y un porcentaje de relacion de 88.50%
# Con 150 neuronas, 100 epochs y 10 de batch_size (lr=0.04) logre una perdida de 50,20% y un porcentaje de relacion de 89.56%

# Con 150 neuronas, 200 epochs y 10 de batch_size (lr=0.03) logre una perdida de 39,36% y un porcentaje de relacion de 93.00%
# Con 150 neuronas, 200 epochs y 50 de batch_size (lr=0.03) logre una perdida de 42,77% y un porcentaje de relacion de 94.52%
# Con 150 neuronas, 200 epochs y 20 de batch_size (lr=0.03) logre una perdida de 36,48% y un porcentaje de relacion de 96.08%
# Con 150 neuronas, 200 epochs y 10 de batch_size (lr=0.03) logre una perdida de 39,78% y un porcentaje de relacion de 93.70%
# Con 150 neuronas, 200 epochs y 30 de batch_size (lr=0.03) logre una perdida de 38.03% y un porcentaje de relacion de 95.96%
# Con 150 neuronas, 200 epochs y 20 de batch_size (lr=0.03) logre una perdida de 34.10% y un porcentaje de relacion de 96.26% (Añadiendo una capa extra)
# Con 150 neuronas, 300 epochs y 20 de batch_size (lr=0.03) logre una perdida de 30.50% y un porcentaje de relacion de 97.80% (Añadiendo una capa extra) (Tiempo: 33 min)
# Con 200 neuronas, 300 epochs y 20 de batch_size (lr=0.03) logre una perdida de 21.50% y un porcentaje de relacion de 99.30% (Añadiendo una capa extra) (Tiempo: 42 min)
# Con 200 neuronas, 350 epochs y 20 de batch_size (lr=0.03) logre una perdida de 20.44% y un porcentaje de relacion de 98.17% con 10k datos (Añadiendo una capa extra) (Tiempo: 49 min)
# Con 200 neuronas, 300 epochs y 20 de batch_size (lr=0.03) logre una perdida de 18.93% y un porcentaje de relacion de 98.71% con 10k datos (Añadiendo una capa extra) (Tiempo: 32 min)
# Con 200 neuronas, 300 epochs y 20 de batch_size (lr=0.03) logre una perdida de 21.28% y un porcentaje de relacion de 98.19% con 10k datos (Añadiendo una capa extra) (Tiempo: 24 min)


Epoch 50/300, Loss: 0.5501
Epoch 100/300, Loss: 0.3938
Epoch 150/300, Loss: 0.3098
Epoch 200/300, Loss: 0.2617
Epoch 250/300, Loss: 0.2320
Epoch 300/300, Loss: 0.2128


In [56]:
# En este bloque de codigo llevamos a cabo las pruebas de nuestro modelo ya entrenado
# Para estas pruebas usamos los ultimos 10000 datos que tenemos dentro de nuestra variable X
X_test = X[60000:, :].copy()

# Se normalizan los datos de prueba si es necesario (utilizando la misma normalización que con los datos de entrenamiento) (media y desviacion estandar)
X_test_norm = (X_test - X_mean) / X_std

# Hacemos uso del modelo que entrenamos pasando los datos de prueba a través del modelo para obtener las predicciones
y_pred_test = mlp(X_test_norm)

# Convierte las predicciones a clases
prediccion = np.argmax(y_pred_test, axis=1)

# Imprimimos las predicciones hechas por nuestro modelo para verificar
print("Predicciones del modelo:")
print(prediccion)
print("Valores reales:")
print(Y[60000:])

# Calculamos el porcentaje de relación que hay entre las predicciones y los valores reales
total_ejemplos = len(prediccion)
correlacion = np.sum(prediccion == Y[60000:])
porcentaje = (correlacion / total_ejemplos) * 100

# Mostramos el porcentaje de relacion que obtenemos con nuestro modelo entrenado
print("Porcentaje de relación:")
print(f"{porcentaje:.2f}%")

Predicciones del modelo:
[1 0 0 ... 2 1 1]
Valores reales:
[1. 0. 0. ... 2. 1. 1.]
Porcentaje de relación:
98.19%
